# IBM Drain3 Demo

Logs are an effective source of data to understand system health and behavior. However, they can often be long and noisy, which makes it difficult for a human to manually analyze them.

In 2017, the [LogPAI](https://github.com/logpai) team developed an online log parsing algorithm called [Drain](https://github.com/logpai/logparser/blob/e8d96cd4de1121c5d2b517982c6028cd06e643f1/docs/tools/Drain.md), that parses logs to extract "log templates" from them. Using these log templates instead of raw logs makes it much easier and faster to analyze log data. The IBM research team then upgraded the official implementation of this project, added some new features and bugfixes, and wrapped it into a python package called [drain3](https://github.com/IBM/Drain3).

In this notebook, we will demonstrate how you can use the IBM drain3 library can be to extract log templates from raw logs data. 

In [1]:
import requests
from drain3 import TemplateMiner

In [2]:
# download sample log file
url = 'https://raw.githubusercontent.com/logpai/loghub/master/Apache/Apache_2k.log'
ret = requests.get(url)
ret.status_code

200

In [3]:
# read the log file and convert to list of log lines
log_file = ret.content.decode("utf-8")
log_lines = log_file.split(sep='\n')

# split into two sets for "training" and "testing"
train_lines = log_lines[:1900]
test_lines = log_lines[-100:]

# how many lines in the log file
print(f'There are {len(log_lines)} lines in the log file. Some of them are:', end='\n\n')

# peek at the first 10 lines in the log
print('\n'.join(log_lines[:10]))

There are 2000 lines in the log file. Some of them are:

[Sun Dec 04 04:47:44 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Dec 04 04:47:44 2005] [error] mod_jk child workerEnv in error state 6
[Sun Dec 04 04:51:08 2005] [notice] jk2_init() Found child 6725 in scoreboard slot 10
[Sun Dec 04 04:51:09 2005] [notice] jk2_init() Found child 6726 in scoreboard slot 8
[Sun Dec 04 04:51:09 2005] [notice] jk2_init() Found child 6728 in scoreboard slot 6
[Sun Dec 04 04:51:14 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Dec 04 04:51:14 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Dec 04 04:51:14 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
[Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6
[Sun Dec 04 04:51:18 2005] [error] mod_jk child workerEnv in error state 6


## Perform Template Mining with Default Configuration

In this section, we will use drain to mine the log templates from the first 1900 line of the log file.

The drain [research paper](http://jiemingzhu.github.io/pub/pjhe_icws2017.pdf) as well as the IBM drain3 repo [readme](https://github.com/IBM/Drain3#configuration) explains the parameters that the drain log parser can be configured with. In this section, we will simply run it with the default parameters. And try to find the log templates that exist in our sample log file.

In [5]:
# init drain3 template miner and view config
miner = TemplateMiner()

print('Drain3 config:\n')
for k,v in vars(miner.config).items():
    print(f'{k} = {v}')

Drain3 config:

profiling_enabled = False
profiling_report_sec = 60
snapshot_interval_minutes = 5
snapshot_compress_state = True
drain_extra_delimiters = []
drain_sim_th = 0.4
drain_depth = 4
drain_max_children = 100
drain_max_clusters = None
masking_instructions = []
mask_prefix = <
mask_suffix = >


In [6]:
# mine templates
# notice that the lines are added one by one, as a "stream"
for line in train_lines:
    miner_result = miner.add_log_message(line)

In [7]:
# what are templates mined in this logfile
for c in miner.drain.clusters:
    print(c, end='\n\n')

ID=1     : size=298       : [Sun Dec 04 <*> 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties

ID=2     : size=281       : [Sun Dec 04 <*> 2005] [error] mod_jk child workerEnv in error state <*>

ID=3     : size=442       : [Sun Dec 04 <*> 2005] [notice] jk2_init() Found child <*> in scoreboard slot <*>

ID=4     : size=18        : [Sun Dec 04 <*> 2005] [error] [client <*> Directory index forbidden by rule: /var/www/html/

ID=5     : size=6         : [Sun Dec 04 <*> 2005] [error] jk2_init() Can't find child <*> in scoreboard

ID=6     : size=6         : [Sun Dec 04 <*> 2005] [error] mod_jk child init 1 -2

ID=7     : size=12        : [Mon Dec 05 <*> 2005] [error] [client <*> Directory index forbidden by rule: /var/www/html/

ID=8     : size=360       : [Mon Dec 05 <*> 2005] [notice] jk2_init() Found child <*> in scoreboard slot <*>

ID=9     : size=239       : [Mon Dec 05 <*> 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties

ID=10    : size=22

**RESULT** From the above output, we can see that drain has learned 12 templates from the log. That is, there are only 12 unique general "structures" or "types" of log lines in our file. So to understand the system behavior, we can simply look at these 12 templates instead of looking at 2000 raw log lines. In this way, drain can significantly reduce the time and effort spent in log file analysis.

However, we can see that it could not identify the timestamps at the beginning of each log line as template wildcard items. So in the next sections, we will try to explore how we can update the configuration to further improve template mining.

## Perform Template Mining with Custom Configurations

### Masking Config
In this section, we'll run drain with a custom "masking" parameter. Since we know what the timestamps look like, we can add a regex to "mask" the timestamps so that they do not count as a part of the template learning.

To do that, we will create a `drain3.ini` file in the working directory with the following section in it:
```
[MASKING]
masking = [
          {"regex_pattern": "\\[(.*?)2005\\]", "mask_with": "TIMESTAMP"}
          ]
```
Notice that here we're just adding one custom regex, but in general you can add a list of regex'es to mask known patterns in log files, as shown in the example [here](https://github.com/IBM/Drain3/blob/15470e391caed9a9ef5038cdd1dbd373bd2386a8/examples/drain3.ini).

In [14]:
# init drain3 template miner with new masking config
miner = TemplateMiner()

print('Drain3 config:\n')
for k,v in vars(miner.config).items():
    print(f'{k} = {v}')

Drain3 config:

profiling_enabled = False
profiling_report_sec = 60
snapshot_interval_minutes = 5
snapshot_compress_state = True
drain_extra_delimiters = []
drain_sim_th = 0.4
drain_depth = 4
drain_max_children = 100
drain_max_clusters = None
masking_instructions = [<drain3.masking.MaskingInstruction object at 0x7fe3af09be48>]
mask_prefix = <
mask_suffix = >


In [15]:
# mine templates
for line in train_lines:
    miner_result = miner.add_log_message(line)

In [16]:
# what are templates mined in this logfile
for c in miner.drain.clusters:
    print(c, end='\n\n')

ID=1     : size=537       : <TIMESTAMP> [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties

ID=2     : size=507       : <TIMESTAMP> [error] mod_jk child workerEnv in error state <*>

ID=3     : size=802       : <TIMESTAMP> [notice] jk2_init() Found child <*> in scoreboard slot <*>

ID=4     : size=30        : <TIMESTAMP> [error] [client <*> Directory index forbidden by rule: /var/www/html/

ID=5     : size=12        : <TIMESTAMP> [error] jk2_init() Can't find child <*> in scoreboard

ID=6     : size=12        : <TIMESTAMP> [error] mod_jk child init 1 -2



**RESULT** Great, with the new masking config we were able to generalize the log templates even more. That is, the previous separate templates such as `[Mon Dec 05 <*> 2005] [error] mod_jk child init 1 -2` and `[Sun Dec 04 <*> 2005] [error] mod_jk child init 1 -2`, that differed in just the timestamps, now got clubbed as one single template, `[Sun Dec 04 <*> 2005] [error] mod_jk child init 1 -2`.

Thus, now instead of looking at 2000 log lines, we only need to look at 6 log lines to understand system behavior characterized by this log file.

### Drain3 Config

In this section, we'll run drain with a custom "drain3" parameter called `similarity threshold`. This parameter dictates how similar a log line has to be to a given template, to be considered as belonging to the same template.

To do that, we will add the following section to our `drain3.ini` file:
```
[DRAIN]
sim_th = 0.2
```

In [11]:
# init drain3 template miner with added drain3 parameter
miner = TemplateMiner()

print('Drain3 config:\n')
for k,v in vars(miner.config).items():
    print(f'{k} = {v}')

Drain3 config:

profiling_enabled = False
profiling_report_sec = 60
snapshot_interval_minutes = 5
snapshot_compress_state = True
drain_extra_delimiters = []
drain_sim_th = 0.2
drain_depth = 4
drain_max_children = 100
drain_max_clusters = None
masking_instructions = [<drain3.masking.MaskingInstruction object at 0x7fe3af4c4da0>]
mask_prefix = <
mask_suffix = >


In [12]:
# mine templates
for line in train_lines:
    miner_result = miner.add_log_message(line)

In [13]:
# what are templates mined in this logfile
for c in miner.drain.clusters:
    print(c, end='\n\n')

ID=1     : size=537       : <TIMESTAMP> [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties

ID=2     : size=519       : <TIMESTAMP> [error] <*> <*> <*> <*> <*> <*> <*>

ID=3     : size=802       : <TIMESTAMP> [notice] jk2_init() Found child <*> in scoreboard slot <*>

ID=4     : size=30        : <TIMESTAMP> [error] [client <*> Directory index forbidden by rule: /var/www/html/

ID=5     : size=12        : <TIMESTAMP> [error] mod_jk child init 1 -2



**RESULT** As we can see, setting the similarity threshold to a low value resulted in merging of two distinct log templates into one. Thus, arbitrarily updating the similarity threshold without properly understanding the domain can yield worse results.

The default drain parameters are set based on the research authors' experiments, so they should generally work fine, but can also be tuned for optimal performance as shown in the above two sections.

## Match New Logs to Mined Templates (Inference)

In the previous sections, we used the first 1900 lines in our logfile to mine the templates. Now we will use the last 100 lines to run inference. That is, we will match / bucket each unseen log line to one of the learned log templates.

In [17]:
# assign templates to new log lines
results = []
for line in test_lines:
    results.append(miner.match(line))

num_egs_to_display = 10
for i, (line, template) in enumerate(zip(log_lines[-100:], results)):
    print(line)
    print(template.get_template())
    print()

    if i > num_egs_to_display:
        break

[Mon Dec 05 17:40:38 2005] [notice] jk2_init() Found child 6577 in scoreboard slot 7
<TIMESTAMP> [notice] jk2_init() Found child <*> in scoreboard slot <*>

[Mon Dec 05 17:40:38 2005] [notice] jk2_init() Found child 6578 in scoreboard slot 8
<TIMESTAMP> [notice] jk2_init() Found child <*> in scoreboard slot <*>

[Mon Dec 05 17:40:39 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
<TIMESTAMP> [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties

[Mon Dec 05 17:40:39 2005] [error] mod_jk child workerEnv in error state 6
<TIMESTAMP> [error] mod_jk child workerEnv in error state <*>

[Mon Dec 05 17:40:39 2005] [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties
<TIMESTAMP> [notice] workerEnv.init() ok /etc/httpd/conf/workers2.properties

[Mon Dec 05 17:40:39 2005] [error] mod_jk child workerEnv in error state 6
<TIMESTAMP> [error] mod_jk child workerEnv in error state <*>

[Mon Dec 05 17:46:02 2005] [notice] jk2_init() Found child 6585 in sco